# 2.2 Ітераційні методи розв'язування СЛАР
---------------------

## 2.2.4.  Метод Зейделя

Нехай маємо СЛАР

$(1)\qquad\qquad\qquad A\,x = b,$

де  $x \in \mathbb{R}^{n}$  --- невідомий вектор, $n \ge 2$ - довільне натуральне число,
$A\in \mathbb{R}^{n\times n},\; b\in \mathbb{R}^{n}$ - задані квадратна матриця і вектор вільних членів відповідно:

$(2)\qquad\qquad
A=
\begin{pmatrix}
a_{11} & a_{12} &\ldots & a_{1n} \\
a_{21} & a_{22} &\ldots & a_{2n} \\
\ldots & \ldots & \ldots \\
a_{n1} & a_{n2} &\ldots & a_{nn}
\end{pmatrix},\quad  
b=
\begin{pmatrix}
b_{1}\\
b_{2}\\
\ldots  \\
b_{n}
\end{pmatrix}.
$

Вважаємо, що $\mathrm{det} A\neq 0$, а також $a_{ii} \neq 0, \; i \in \overline {1, n}$.


Метод *Зейделя* полягає в обчисленні послідовності 

$\qquad\qquad x^0, x^1, \ldots, x^k, \ldots$

наближень розв'язку системи за правилом:

$(3)\qquad\qquad\qquad
x_{i}^{k+1} =  - \sum_{j=1}^{i-1} \frac{a_{ij}}{a_{ii}} x_j^{k+1} - \sum_{j=i+1}^n
\frac{a_{ij}}{a_{ii}} x_j^k + d_i,\quad
 i =\overline{1,n},\ \ \ k\in \mathbb{N}_0.
$

де 
$x^k =(x_1^k,x_2^k, \dots , x_n^k)^\top$ --- $k$-та ітерація вектора наближень, $x^0 \in \mathbb{R}^n$ --- початкове наближення.


#### Пояснення до використання програмного коду
-----------------

*   Підготувати  функції, які потрібні для реалізації методу: 
    1.   виконати комірку з імпортом ``NumPy``
    2.   виконати комірку з визначеннями допоміжних функцій ``norm_1``,``norm_2`` i ``norm_3``    
    3.   виконати комірку з визначеннями функції ``simple_iteration`` 
    4.   виконати комірку з визначеннями функцій ``Jacobi_modification`` і ``Seidel_solver``
*   Підготувати  функції, які задають конкретну СЛАР і дані чисельного методу
    1.   виконати комірки з визначеннями функцій ``set_matrix`` i ``set_vector`` 
    2.   виконати комірку з визначеннями функцій ``set_x0`` 
    3.   виконати комірку із заданням точності ``eps`` і максимальної кількості ітерацій ``max_iter``
*   Виконати комірку з викликом функції ``Seidel_solver``

In [ ]:
import numpy as np

In [ ]:
# Допоміжні функції

def norm_1(a):
    """обчислення норми_1 матриці a"""   
    m=0
    for i in range(a.shape[0]):
        s=0
        for j in range(a.shape[1]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m

def norm_2(a):
    """обчислення норми_2 матриці a"""   
    m=0
    for j in range(a.shape[1]):
        s=0
        for i in range(a.shape[0]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m

def norm_3(a):
    """обчислення евклідової норми вектора a"""
    return np.sqrt(np.sum(a**2))

>#### ``Jacobi_modification`` модифікує матрицю і вектор вільних членів СЛАР (1)     

-------------------

In [ ]:
def Jacobi_modification(a, b):
    """ модифікація матриці і вектора вільних членів СЛАР """
    for i in range(a.shape[0]):
        b[i] /= a[i,i]
        a[i,:] /= -a[i,i]
        a[i,i]=0
        

>### Функція  ``Seidel_iteration``  реалізує метод Зейделя для розв'язування СЛАР

-------------------
*   На кожній ітерації $k$ для зберігання попереднього наближення розв'язку $x^{k-1}$ використовуємо вектор x_prev, 
*   Після обчислення нового значення $x^{k}$ зберігаємо його у векторі x_new  
    



In [ ]:
def Seidel_iteration(b,d,x0,eps,iter_print):
    """ послідовно обчислює наближення розв'язку СЛАР 
      за методом Зейделя, поки евклідова норма 
      різниці двох послідовних наближень не стане меншою заданої точності eps      
      """
    n=b.shape[0]
    x_prev=x0.copy()
    
    k=1
    x_n = np.empty(n) 
    x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]
    for i in range(1,n):
        x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 

    while norm_3(x_n - x_prev)  > eps and k < max_iter:        
        x_prev=x_n.copy()  
        
        k+=1
        x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]   
        for i in range(1,n):
            x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 
            
    return k, x_n
    

>#### ``Seidel_solver`` організовує обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps`` 
-------------------


In [ ]:
def Seidel_solver(set_matrix, set_vector,set_x0, eps, max_iter):
    """ Розв'язування СЛАР методом простих ітерацій """      
    b=set_matrix()  
    d=set_vector()
    Jacobi_modification(b, d) 
    k=0
    if norm_1(b)<1 or norm_2(b)<1 : 
        x0=set_x0()        
        k, x = Seidel_iteration(b,d,x0,eps,max_iter)
        return k, x   
    else:
        return k, d 
    

>#### Обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps``

-------------------

In [ ]:
def set_matrix():
    """ функція для задання матриці СЛАР"""
    matrix=np.array([[10, 1, 2],[1,5,-1],[1,-2,10]],dtype=float )
    return matrix

In [ ]:
def set_vector():
    """ функція для задання вектора вільних членів СЛАР"""
    vector=np.array([[18], [8], [27]],dtype=float)
    return vector

In [ ]:
def set_x0():
    """ функція для задання вектора початкового наближення розв'язку"""
    return np.array([[1], [2], [1]],dtype=float)

In [ ]:
eps=0.001
max_iter=100

In [ ]:
cond, x = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if cond > 0 and cond < max_iter:
    print(f"Наближений розв'язок системи \n {x} \n обчислено за {cond} ітерацій")
elif  cond == max_iter : 
    print(f"Наближений розв'язок системи не обчислено за {cond} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")